In [1]:
# Importa las librerias necesarias
import pandas as pd
import numpy as np
import os

In [2]:
# Carga el archivo JSON en un DataFrame
steam_games = pd.read_json('../datasets/output_steam_games.json', lines= True)

steam_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,js41637,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0


In [3]:
# Devuelve la cantidad de nulos que se encuentran en cada columna del Dataframe
steam_games.isnull().sum()

publisher          96362
genres             91593
app_name           88312
title              90360
url                88310
release_date       90377
tags               88473
reviews_url        88312
discount_price    120220
specs              88980
price              89687
early_access       88310
id                 88312
metascore         117768
developer          91609
user_id            32135
steam_id           32135
items              32135
items_count        32135
dtype: int64

In [4]:
# Elimina las filas del DataFrame steam_games que no se consideran necesarias
steam_games = steam_games.iloc[88310:]
# Elimina las columnas que no se consideran necesarias
steam_games = steam_games.drop(columns=['publisher','app_name','url','tags','reviews_url','discount_price','early_access','metascore','user_id','steam_id','items','items_count'])
steam_games.head(2)

,genres,title,release_date,specs,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,[Single-player],4.99,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,643980.0,Secret Level SRL


In [5]:
# Utiliza la funcion explode en la columna 'genres' para convertir los valores anidados en filas separadas
steam_games = steam_games.explode('genres')
# Restablecer el índice del DataFrame resultante y descartar el índice anterior
steam_games = steam_games.reset_index(drop=True)
steam_games.head(2)

,genres,title,release_date,specs,price,id,developer
0,Action,Lost Summoner Kitty,2018-01-04,[Single-player],4.99,761140.0,Kotoshiro
1,Casual,Lost Summoner Kitty,2018-01-04,[Single-player],4.99,761140.0,Kotoshiro


In [6]:
# Utiliza la funcion explode en la columna 'specs' para convertir los valores anidados en filas separadas
steam_games = steam_games.explode('specs')
# Restablecer el índice del DataFrame resultante y descartar el índice anterior
steam_games = steam_games.reset_index(drop=True)
steam_games.head(2)

,genres,title,release_date,specs,price,id,developer
0,Action,Lost Summoner Kitty,2018-01-04,Single-player,4.99,761140.0,Kotoshiro
1,Casual,Lost Summoner Kitty,2018-01-04,Single-player,4.99,761140.0,Kotoshiro


In [7]:
# Elimina valores nulos de la columna genres
steam_games = steam_games.dropna(subset=['genres'])
# Elimina valores nulos de la columna id
steam_games = steam_games.dropna(subset=['id'])
# Elimina valores nulos de la columna specs
steam_games = steam_games.dropna(subset=['specs'])

In [8]:
# Renombra la columna 'title' a 'item_name'
steam_games = steam_games.rename(columns={'title': 'item_name'})

In [9]:
# Reemplaza los valores 'Soon..' en la columna 'release_date' con NaN (valores nulos)
steam_games["release_date"] = steam_games["release_date"].replace("Soon..", np.nan)
# Convierte la columna 'release_date' a tipo de dato de fecha
steam_games['release_date'] = pd.to_datetime(steam_games['release_date'], errors='coerce', format='%Y-%m-%d')

# Define las columnas que deseas llenar con la moda (en este caso, solo 'release_date')
columns_fill_mode = ['release_date']
for col in columns_fill_mode:
    steam_games[col].fillna(steam_games[col].mode()[0], inplace=True)

# Extraer el año y almacenarlo en una nueva columna 'release_year'
steam_games['release_year'] = steam_games['release_date'].dt.year.astype('Int64')

steam_games.head(2)

,genres,item_name,release_date,specs,price,id,developer,release_year
0,Action,Lost Summoner Kitty,2018-01-04,Single-player,4.99,761140.0,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,2018-01-04,Single-player,4.99,761140.0,Kotoshiro,2018


In [10]:
# Elimina la columna 'release_date'
steam_games = steam_games.drop(columns='release_date')

In [11]:
# Muestra los nulos del Dataframe
steam_games.isnull().sum()

genres              0
item_name           4
specs               0
price           12970
id                  0
developer        1358
release_year        0
dtype: int64

In [12]:
# Convierte la columna "price" a valores numéricos y establece 0 para los valores que no se pueden convertir
steam_games['price'] = pd.to_numeric(steam_games['price'], errors='coerce').fillna(0)

In [13]:
# Muestra los nulos del Dataframe
steam_games.isnull().sum()

genres             0
item_name          4
specs              0
price              0
id                 0
developer       1358
release_year       0
dtype: int64

In [14]:
# Convierte la columna 'id' a tipo de dato int
steam_games['id'] = steam_games['id'].astype(int)
# Renombra la columna 'id' como 'item_id'
steam_games = steam_games.rename(columns={'id': 'item_id'})

In [15]:
# Convertir la columna 'item_name' a tipo de dato str
steam_games['item_name'] = steam_games['item_name'].astype(str)

In [16]:
# Llenar los valores nulos en la columna 'item_name' con un espacio vacío ('')
steam_games['item_name'] = steam_games['item_name'].fillna('')

In [17]:
# Guarda el DataFrame steam_games en un archivo Parquet
steam_games.to_parquet(os.path.join('../datasets/', 'steam_games.parquet'), index=False)